In [5]:
import pandas as pd
import numpy as np
import warnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Ignore expected forecasting warnings
warnings.filterwarnings("ignore")


In [6]:
# Load raw CSV
df = pd.read_csv("vodafone_churn_kpi_raw_2025.csv")

# Standardize column names
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

df.head()


,ingestion_date,product_group,footprint,customer_type,churn_value,reporting_kpi
0,29-09-2025,Broadband,KD,Bad_Payer,8,CHURN_COUNT
1,12-05-2025,DSL,DSL,Regular,20,CHURN_COUNT
2,16-06-2025,Broadband,UM,Regular,18,CHURN_COUNT
3,16-06-2025,Fiber,Fiber,Regular,20,CHURN_COUNT
4,26-05-2025,Broadband,UM,Bad_Payer,4,CHURN_COUNT


In [7]:
## 2. Schema Normalization
# Normalize customer type column
for col in ["customer_type", "churn_classification", "churn_type", "customer_segment"]:
    if col in df.columns:
        df = df.rename(columns={col: "customer_type"})
        break

# Normalize churn value column
for col in ["churn_value", "actual_values", "actual_value", "value"]:
    if col in df.columns:
        df = df.rename(columns={col: "raw_churn_value"})
        break


In [8]:
## 3. Business Rules

#- Raw data represents **ACTUAL churn only**
#- Any ingestion date from 2026 onwards is excluded
#- Reporting is done at a **monthly level**


In [9]:
# Parse ingestion date
df["ingestion_date"] = pd.to_datetime(df["ingestion_date"], dayfirst=True)

# Keep only valid actuals
df = df[df["ingestion_date"] < "2026-01-01"]

# Create monthly reporting bucket (month start)
df["reporting_month"] = (
    df["ingestion_date"]
    .dt.to_period("M")
    .dt.to_timestamp()
    + pd.offsets.MonthBegin(0)
)


In [10]:
## 4. Actual Churn Reporting Layer

#Actual churn is aggregated monthly and never modified.
actuals = (
    df.groupby(
        [
            "reporting_month",
            "ingestion_date",
            "product_group",
            "footprint",
            "customer_type"
        ],
        as_index=False
    )
    .agg(churn_value=("raw_churn_value", "sum"))
)

actuals["reporting_kpi"] = "ACTUAL"
actuals["forecast_date"] = pd.NaT

actuals.head()



,reporting_month,ingestion_date,product_group,footprint,customer_type,churn_value,reporting_kpi,forecast_date
0,2025-01-01,2025-01-06,Broadband,KD,Bad_Payer,13,ACTUAL,NaT
1,2025-01-01,2025-01-06,Broadband,KD,Regular,78,ACTUAL,NaT
2,2025-01-01,2025-01-06,Broadband,UM,Regular,23,ACTUAL,NaT
3,2025-01-01,2025-01-06,DSL,DSL,Bad_Payer,9,ACTUAL,NaT
4,2025-01-01,2025-01-06,DSL,DSL,Move,10,ACTUAL,NaT


In [11]:
## 5. Rolling 3-Month Churn Forecast

#For each weekly ingestion snapshot:
#- Historical actuals are frozen
#- A 3-month forecast is generated
#- Exponential Smoothing (ETS) is used

forecast_rows = []
dimension_keys = ["product_group", "footprint", "customer_type"]

for ingestion_date in sorted(actuals["ingestion_date"].unique()):

    ingestion_month = (
        pd.Timestamp(ingestion_date)
        .to_period("M")
        .to_timestamp()
        + pd.offsets.MonthBegin(0)
    )

    snapshot = actuals[actuals["reporting_month"] <= ingestion_month]

    for dims, group in snapshot.groupby(dimension_keys):

        ts = (
            group.groupby("reporting_month")["churn_value"]
            .sum()
            .sort_index()
        )

        ts.index = pd.DatetimeIndex(ts.index)
        ts = ts.asfreq("MS")

        if len(ts.dropna()) < 4:
            continue

        model = ExponentialSmoothing(
            ts,
            trend="add",
            seasonal=None,
            initialization_method="estimated"
        ).fit()

        preds = (
            model.forecast(3)
            .fillna(0)
            .clip(lower=0)
            .round()
            .astype(int)
        )

        future_months = pd.date_range(
            start=ingestion_month + pd.offsets.MonthBegin(1),
            periods=3,
            freq="MS"
        )

        for m, v in zip(future_months, preds):
            forecast_rows.append({
                "reporting_month": m,
                "ingestion_date": ingestion_date,
                "product_group": dims[0],
                "footprint": dims[1],
                "customer_type": dims[2],
                "churn_value": v,
                "reporting_kpi": "FORECAST",
                "forecast_date": m
            })



In [12]:
## 6. Final Reporting Dataset

#Actuals and forecasts are combined into a single reporting table.
forecast_df = pd.DataFrame(forecast_rows)

final_df = (
    pd.concat([actuals, forecast_df], ignore_index=True)
    .sort_values(["ingestion_date", "reporting_month"])
)

final_df.to_csv("churn_forecast_reporting.csv", index=False)

final_df.head()


,reporting_month,ingestion_date,product_group,footprint,customer_type,churn_value,reporting_kpi,forecast_date
0,2025-01-01,2025-01-06,Broadband,KD,Bad_Payer,13,ACTUAL,NaT
1,2025-01-01,2025-01-06,Broadband,KD,Regular,78,ACTUAL,NaT
2,2025-01-01,2025-01-06,Broadband,UM,Regular,23,ACTUAL,NaT
3,2025-01-01,2025-01-06,DSL,DSL,Bad_Payer,9,ACTUAL,NaT
4,2025-01-01,2025-01-06,DSL,DSL,Move,10,ACTUAL,NaT
